# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management. There are actually more than 500 stocks because companies like Google has multiple segements

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import time

In [2]:
def fetch_data(stock):
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
# we want endpoints that contain market capitalization and price
#api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
print(data)

[{'avgTotalVolume': 53551384, 'calculationPrice': 'close', 'change': -6.89, 'changePercent': -0.03579, 'close': 185.64, 'closeSource': 'official', 'closeTime': 1704229200398, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 185.575, 'delayedPriceTime': 1704229190718, 'extendedChange': -0.44, 'extendedChangePercent': -0.00237, 'extendedPrice': 185.2, 'extendedPriceTime': 1704243596499, 'high': 188.44, 'highSource': '15 minute delayed price', 'highTime': 1704229199995, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 185.55, 'iexCloseTime': 1704229199917, 'iexLastUpdated': 1704229199917, 'iexMarketPercent': 0.02179722273145038, 'iexOpen': 187.19, 'iexOpenTime': 1704205800254, 'iexRealtimePrice': 185.55, 'iexRealtimeSize': 200, 'iexVolume': 1798024, 'lastTradeTime': 1704229199930, 'latestPrice': 185.64, 'latestSource': 'Close', 'latestTime': 'January 2, 2024', 'latestUpdate': 1704229200398, 'latestVolume': 82488674, 'low': 183.885, 'lowSource

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data[0]['latestPrice']
market_cap = data[0]['marketCap']
if type(market_cap) == int:
    market_cap = market_cap/1000000000000


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
row_to_add = pd.DataFrame(
    [[symbol, price, market_cap, 'N/A']], 
    columns = my_columns
)

final_dataframe = pd.concat(
    [final_dataframe, row_to_add], 
    ignore_index=True
)
final_dataframe

/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_32273/3925092863.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,185.64,2.887213,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
total_requests = 500
requests_made = 0

while requests_made < total_requests:
    for stock in stocks['Ticker']:
        data = fetch_data(stock)
        if data:
            price = data[0]['latestPrice']
            market_cap = data[0]['marketCap']
            if type(market_cap) == int:
                market_cap = market_cap/1000000000000

            row_to_add = pd.DataFrame(
                [[stock, price, market_cap, 'N/A']], 
                columns = my_columns
            )

            final_dataframe = pd.concat(
                [final_dataframe, row_to_add], 
                ignore_index=True
            )

        requests_made += 1
        if requests_made % 25 == 0:
            print("Reached 25 requests, waiting for 60 seconds.")
            time.sleep(60)  # Wait for 1 minute after every 20 requests

        if requests_made >= total_requests:
            break

In [ ]:
# add remaining four
for stock in stocks['Ticker'][496:506]:
    data = fetch_data(stock)
    if data and data[0] is not None:
        price = data[0]['latestPrice']
        market_cap = data[0]['marketCap']
        if type(market_cap) == int:
            market_cap = market_cap/1000000000000

        row_to_add = pd.DataFrame(
            [[stock, price, market_cap, 'N/A']], 
            columns = my_columns
        )

        final_dataframe = pd.concat(
            [final_dataframe, row_to_add], 
            ignore_index=True
        )
final_dataframe

In [14]:
#Output the final_dataframe to a csv file
final_dataframe.to_csv('final_dataframe.csv', index=False)

In [ ]:
#Read in final_dataframe from final_dataframe.csv
final_dataframe = pd.read_csv('final_dataframe_project1.csv')
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for item in data:  # Iterate over each dictionary in the list
        symbol = item['symbol']  # Access the symbol directly from the dictionary
        price = item['latestPrice']
        market_cap = item['marketCap']
        
        '''
        # Check if market_cap is an integer (though it should be given the data structure)
        if isinstance(market_cap, int):
            market_cap = market_cap / 1000000000000  # Adjust market cap as necessary
        '''    

        # Your existing code for adding a row to the DataFrame
        row_to_add = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns=my_columns)
    
        final_dataframe = pd.concat([final_dataframe, row_to_add], ignore_index=True)
        
final_dataframe
    
    


/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_32273/3456477492.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, row_to_add], ignore_index=True)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.75,40654319153,N/A
1,AAL,13.44,8783584992,N/A
2,AAP,61.95,3686148281,N/A
3,AAPL,185.64,2887212881280,N/A
4,ABBV,159.82,287676000000,N/A
...,...,...,...,...
500,YUM,129.03,36168169498,N/A
501,ZBH,121.39,25368168508,N/A
502,ZBRA,267.98,13763346680,N/A
503,ZION,44.14,6539287988,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
'''
portfolio_size = input('Enter the value of your portfolio: ')
# if statement under try turns out to cause error, the statement under except will be executed
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again: ")
    portfolio_size = input('Please the value of your portfolio')
    val = float(portfolio_size)

    # A drawback is that if the user enters non integer again, error occurs
    
'''

while True:
    try:
        portfolio_size = input('Enter the value of your portfolio: ')
        val = float(portfolio_size)
        print(val)
        break  # Break out of the loop if the conversion is successful
    except ValueError:
        print("That's not a number! Please try again:")

Enter the value of your portfolio: 
That's not a number! Please try again:
Enter the value of your portfolio: 10000000
10000000.0


In [12]:
position_size = val/len(final_dataframe.index)
number_of_apple_shares = position_size/500
# we can't buy partial share
print(math.floor(number_of_apple_shares))

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    print(final_dataframe.loc[i, 'Number of Shares to Buy'])
    

39
142
1473
319
106
123
110
51
180
57
34
102
272
84
84
267
238
1023
237
287
2528
116
88
169
134
73
516
137
159
108
128
2030
142
121
66
52
90
131
85
56
41
68
243
548
72
204
219
153
169
209
105
18
99
148
105
7
78
584
509
256
80
676
348
73
60
378
5
583
24
284
375
99
54
342
550
274
373
670
192
350
67
87
111
212
169
1145
75
89
127
208
242
590
209
227
50
63
187
245
136
350
453
94
8
82
334
257
682
149
889
53
168
30
1620
442
416
77
392
571
33
1800
266
407
190
244
132
301
408
489
255
49
176
141
140
132
84
218
810
810
3431
146
138
130
357
47
410
122
177
202
185
431
867
160
145
451
99
214
82
273
136
221
206
162
24
318
310
79
401
82
192
370
262
542
155
133
120
1628
126
311
100
343
470
78
529
112
324
173
569
345
488
70
305
703
654
5641
189
987
342
274
76
157
237
296
162
649
549
141
143
143
155
947
156
50
24
548
396
1539
4429
71
57
136
544
243
76
109
273
94
1169
663
404
603
262
1005
103
42
368
122
156
36
92
243
143
302
182
414
32
541
598
185
85
258
288
59
45
75
1115
154
101
346
120
123
674
115
342
1

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [30]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [31]:
background_color = '#0a0a23'
font_color = 'ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

title_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1,
    'bold': True  # This will make the font bold
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [32]:
'''
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, integer_format)
# A:A is the first column and 18 is the column length
writer.close()

# Properly format the column names
writer.sheets['Recommended Trades'].write('A1', 'Ticker', title_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', title_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', title_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', title_format)

'''

"\nwriter.sheets['Recommended Trades'].set_column('A:A', 18, string_format)\nwriter.sheets['Recommended Trades'].set_column('B:B', 18, dollar_format)\nwriter.sheets['Recommended Trades'].set_column('C:C', 18, dollar_format)\nwriter.sheets['Recommended Trades'].set_column('D:D', 18, integer_format)\n# A:A is the first column and 18 is the column length\nwriter.close()\n\n# Properly format the column names\nwriter.sheets['Recommended Trades'].write('A1', 'Ticker', title_format)\nwriter.sheets['Recommended Trades'].write('B1', 'Stock Price', title_format)\nwriter.sheets['Recommended Trades'].write('C1', 'Market Capitalization', title_format)\nwriter.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', title_format)\n\n"

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [33]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], title_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [34]:
writer.close()